# Data fetching

In [1]:
import pandas as pd
import requests
 
df = pd.read_csv('full_city_list.txt', sep=";")
df_arg = df[df['Country']=='Argentina']
 
province_lst = []

for i in range(len(df_arg)):
    province_id = int(df_arg.iloc[i]['CityId'])
    url = 'https://worldweather.wmo.int/en/json/' + str(province_id) + '_en.json'
    response = requests.get(url)
    province_json = response.json()
    province_lst.append(province_json)
print(len(province_lst))

10


In [2]:
province_lst[0]

{'city': {'lang': 'en',
  'cityName': 'Bariloche',
  'cityLatitude': '-41.150000000',
  'cityLongitude': '-71.170000000',
  'cityId': 858,
  'isCapital': False,
  'stationName': 'Bariloche',
  'tourismURL': '',
  'tourismBoardName': '',
  'isDep': False,
  'timeZone': '-0300',
  'isDST': 'N',
  'member': {'memId': 50,
   'memName': 'Argentina',
   'shortMemName': '',
   'url': 'www.smn.gov.ar',
   'orgName': 'Servicio Meteorológico Nacional',
   'logo': 'argentina_logo.png',
   'ra': 3},
  'forecast': {'issueDate': '2025-04-19 05:59:00',
   'timeZone': 'Local',
   'forecastDay': [{'forecastDate': '2025-04-19',
     'wxdesc': '',
     'weather': 'Partly Cloudy',
     'minTemp': '1',
     'maxTemp': '13',
     'minTempF': '34',
     'maxTempF': '55',
     'weatherIcon': 2202},
    {'forecastDate': '2025-04-20',
     'wxdesc': '',
     'weather': 'Rain',
     'minTemp': '3',
     'maxTemp': '15',
     'minTempF': '37',
     'maxTempF': '59',
     'weatherIcon': 1401},
    {'forecastDate':

# Main Dataframe

In [3]:
# import pandas as pd 
# df = pd.read_json('Cordoba.json')
# df
import pandas as pd
df = pd.json_normalize(province_lst[0])


In [6]:
df

,city.lang,city.cityName,city.cityLatitude,city.cityLongitude,city.cityId,city.isCapital,city.stationName,city.tourismURL,city.tourismBoardName,city.isDep,...,city.climate.datab,city.climate.datae,city.climate.tempb,city.climate.tempe,city.climate.rdayb,city.climate.rdaye,city.climate.rainfallb,city.climate.rainfalle,city.climate.climatefromclino,city.climate.climateMonth
0,en,Bariloche,-41.150000000,-71.170000000,858,False,Bariloche,,,False,...,1981,2010,,,,,,,,"[{'month': 1, 'maxTemp': '22.6', 'minTemp': '6..."


In [5]:
df.columns

Index(['city.lang', 'city.cityName', 'city.cityLatitude', 'city.cityLongitude',
       'city.cityId', 'city.isCapital', 'city.stationName', 'city.tourismURL',
       'city.tourismBoardName', 'city.isDep', 'city.timeZone', 'city.isDST',
       'city.member.memId', 'city.member.memName', 'city.member.shortMemName',
       'city.member.url', 'city.member.orgName', 'city.member.logo',
       'city.member.ra', 'city.forecast.issueDate', 'city.forecast.timeZone',
       'city.forecast.forecastDay', 'city.climate.raintype',
       'city.climate.raindef', 'city.climate.rainunit', 'city.climate.datab',
       'city.climate.datae', 'city.climate.tempb', 'city.climate.tempe',
       'city.climate.rdayb', 'city.climate.rdaye', 'city.climate.rainfallb',
       'city.climate.rainfalle', 'city.climate.climatefromclino',
       'city.climate.climateMonth'],
      dtype='object')

# OBJECT SEGMENTATION

### 3 objects -> [city,member] ; [forecast]; [climate]

In [12]:
# FOR CITY,MEMBER COLUMN OBJECTS

import pandas as pd
from pandas import json_normalize

# Your specified columns
cols = ['city.lang', 'city.cityName', 'city.cityLatitude', 'city.cityLongitude',
       'city.cityId', 'city.isCapital', 'city.stationName', 'city.tourismURL',
       'city.tourismBoardName', 'city.isDep', 'city.timeZone', 'city.isDST',
       'city.member.memId', 'city.member.memName', 'city.member.shortMemName',
       'city.member.url', 'city.member.orgName', 'city.member.logo',
       'city.member.ra']

# nested_cols = ['member']
# selected_df = df[cols].copy()
# selected_df.reset_index(drop=True, inplace=True)
# member_normalized = json_normalize(selected_df['member'])
# member_normalized.reset_index(drop=True, inplace=True)
# selected_df = selected_df.drop(columns=nested_cols)
# city_member = pd.concat([selected_df, member_normalized], axis=1)
city_member_df = df[cols]
city_member_df


,city.lang,city.cityName,city.cityLatitude,city.cityLongitude,city.cityId,city.isCapital,city.stationName,city.tourismURL,city.tourismBoardName,city.isDep,city.timeZone,city.isDST,city.member.memId,city.member.memName,city.member.shortMemName,city.member.url,city.member.orgName,city.member.logo,city.member.ra
0,en,Bariloche,-41.150000000,-71.170000000,858,False,Bariloche,,,False,-0300,N,50,Argentina,,www.smn.gov.ar,Servicio Meteorológico Nacional,argentina_logo.png,3


In [13]:
# FOR CLIMATE COLUMN

import pandas as pd
from pandas import json_normalize
cols=['city.climate.raintype',
       'city.climate.raindef', 'city.climate.rainunit', 'city.climate.datab',
       'city.climate.datae', 'city.climate.tempb', 'city.climate.tempe',
       'city.climate.rdayb', 'city.climate.rdaye', 'city.climate.rainfallb',
       'city.climate.rainfalle', 'city.climate.climatefromclino',
       'city.climate.climateMonth']

climate_df = df[cols]

# Step 1: Extract the climate column
selected_df = df[['city.climate.climateMonth']].copy()

# Step 2: Keep raw values if they're not dicts
selected_df['climate_raw'] = selected_df['city.climate.climateMonth'].apply(
    lambda x: x if not isinstance(x, dict) else None
)

# Step 3: Extract dicts (or empty dicts for non-dict entries — keeps alignment!)
climate_dicts = selected_df['city.climate.climateMonth'].apply(
    lambda x: x if isinstance(x, dict) else {}
)

# Step 4: Normalize top-level fields without dropping rows
climate_top = pd.json_normalize(climate_dicts.tolist(), sep='_')

# Step 5: Flatten climateMonth from list of 12 dicts → wide columns
def flatten_climate_month(month_list):
    month_data = {}
    if isinstance(month_list, list):
        for entry in month_list:
            if isinstance(entry, dict) and 'month' in entry:
                month_name = entry['month']
                for k, v in entry.items():
                    if k != 'month':
                        month_data[f"{k}_{month_name}"] = v
    return month_data

climate_month_data = climate_dicts.apply(
    lambda x: flatten_climate_month(x.get('climateMonth')) if 'climateMonth' in x else {}
)

climate_month_df = pd.DataFrame(climate_month_data.tolist())

# Step 6: Combine everything into a final dataframe
climate_df = pd.concat(
    [selected_df.drop(columns='city.climate.climateMonth'), climate_top.drop(columns='climateMonth', errors='ignore'), climate_month_df],
    axis=1
)
climate_df.drop(columns='climate_raw', inplace=True)
climate_df = climate_df.iloc[1:,:]

climate_df

""


In [ ]:
# FOR FORECAST COLUMN

import pandas as pd
from pandas import json_normalize

# Step 1: Extract the forecast column
forecast_data = df['forecast']

# Step 2: Normalize the forecast dictionary
forecast_dicts = pd.json_normalize(forecast_data)

# Step 3: Flatten the top-level keys
forecast_flat = pd.json_normalize(forecast_dicts.to_dict(orient='records'), sep='_')

# Check what the actual column name for the forecast list is
print(forecast_flat.columns)  # Helps you inspect actual column names

# Most likely, it’s just 'forecastDay'
top_level = forecast_flat.drop(columns=['forecastDay'], errors='ignore')

# Step 4: Function to flatten forecast days using "today", "today+1", ...
def flatten_forecast_by_day_offset(forecast_list):
    flat = {}
    if isinstance(forecast_list, list):
        for i, entry in enumerate(forecast_list):
            label = f"today+{i}" if i > 0 else "today"
            for k, v in entry.items():
                if k != 'forecastDate':  # Skip date, or include if needed
                    flat[f"{k}_{label}"] = v
    return flat

# Step 5: Apply the flattening function to the forecastDay list
flattened_forecasts = forecast_flat['forecastDay'].apply(flatten_forecast_by_day_offset)

# Step 6: Convert list of flattened dicts to DataFrame
forecast_days_df = pd.DataFrame(flattened_forecasts.tolist())

# Step 7: Combine top-level fields with forecast details
forecast_data = pd.concat([top_level, forecast_days_df], axis=1)

# ✅ Final forecast_data DataFrame
forecast_data


Index(['issueDate', 'timeZone', 'forecastDay'], dtype='object')


,issueDate,timeZone,wxdesc_today,weather_today,minTemp_today,maxTemp_today,minTempF_today,maxTempF_today,weatherIcon_today,wxdesc_today+1,...,minTempF_today+5,maxTempF_today+5,weatherIcon_today+5,wxdesc_today+6,weather_today+6,minTemp_today+6,maxTemp_today+6,minTempF_today+6,maxTempF_today+6,weatherIcon_today+6
0,2025-04-06 17:39:00,Local,,Mostly Cloudy,14,20,57,68,2302,,...,55,66,2301,,Fine,13,25,55,77,2501


# 

# rutujas code 